In [1]:
import numpy as np
from joblib import load
from gensim.models import Word2Vec
from sklearn.preprocessing import StandardScaler
import joblib

test_descriptions = [
'книга',
'ковёр',
'чашка', 'картина', 'платье', 'брюки', 'карандаш'
]
# Загрузка модели KNN
knn_model = load('knn_model_100.pkl')

# Загрузка скейлера
scaler = load('scaler.pkl')

# Загрузка модели Word2Vec
w2v_model = load('word2vec.pkl')

# Функция для получения векторов слов
def get_word_vectors(tokens):
    return [w2v_model.wv[word] for word in tokens if word in w2v_model.wv]

# Векторизация данных для тестирования
def vectorize_descriptions(descriptions):
    word_vectors_list = []
    for desc in descriptions:
        tokens = desc.split()  # Разделяем текст на токены
        vectors = get_word_vectors(tokens)
        if vectors:  # Проверяем, что векторы не пустые
            word_vector = np.mean(vectors, axis=0)
            word_vectors_list.append(word_vector)
    # Преобразуем список векторов в массив NumPy
    return np.array(word_vectors_list)

# Применяем векторизацию
test_vectors = vectorize_descriptions(test_descriptions)

# Масштабируем данные
if test_vectors.shape[0] > 0:  # Убедимся, что есть векторы
    test_vectors_scaled = scaler.transform(test_vectors)

    # Делаем предсказания
    predictions = knn_model.predict(test_vectors_scaled)

    # Выводим результаты
    for desc, pred in zip(test_descriptions, predictions):
        print(f"Описание: {desc}")
        print(f"Предсказанный класс: {pred}")
else:
    print("Нет векторов для тестирования. Проверьте входные данные.")

Описание: книга
Предсказанный класс: 61.0
Описание: ковёр
Предсказанный класс: 2.0
Описание: чашка
Предсказанный класс: 4.0
Описание: картина
Предсказанный класс: 44.0
Описание: платье
Предсказанный класс: 58.0
Описание: брюки
Предсказанный класс: 6.0


In [3]:
# Загружаем модель KMeans
kmeans = load('kmeans_100_w2v.pkl')

# Загружаем модель Word2Vec
w2v_model = load('word2vec.pkl')

# Получаем названия всех слов из Word2Vec модели
vocabulary = w2v_model.wv.index_to_key

# Получаем векторы слов
word_vectors = w2v_model.wv[vocabulary]

# Предсказываем метки кластеров для векторов слов
predictions = kmeans.predict(word_vectors)

# Создаём словарь для хранения слов для каждого кластера
clustered_words = {}

# Заполняем словарь с использованием меток кластеров
for index, cluster in enumerate(predictions):
    word = vocabulary[index]
    if cluster not in clustered_words:
        clustered_words[cluster] = []
    clustered_words[cluster].append(word)

# Функция для нахождения главных слов в кластере
def get_top_words(cluster, words):
    vectors_in_cluster = [w2v_model.wv[word] for word in words]
    cluster_center = np.mean(vectors_in_cluster, axis=0)
    
    # Вычисляем расстояние от каждого слова до центра кластера
    distances = {word: np.linalg.norm(w2v_model.wv[word] - cluster_center) for word in words}
    
    # Находим три слова с наименьшим расстоянием до центра
    top_words = sorted(distances, key=distances.get)[:3]
    return top_words

# Выводим 3 основных слова для каждого кластера
for cluster, words in clustered_words.items():
    top_words = get_top_words(cluster, words)
    print(f"Кластер {cluster} - {', '.join(words)}, Главные слова: {', '.join(top_words)}")

Кластер 3 - для, Главные слова: для
Кластер 4 - картина, Главные слова: картина
Кластер 80 - набор, Главные слова: набор
Кластер 5 - фотообои, обликва, саженец, рассада, радардетектор, саквояж, кабачок, видеокарта, луковицы, мастурбатор, мюли, лоферы, казаки, bdre, автокружка, usbконцентратор, ежедневник, сноубутсы, термоперчатки, слипоны, сникеры, окунь, угги, молд, экскурсия, сарай, дутики, фотошторы, офэкт, моноласта, рак, фотоальбом, шлепанцы, электрогирлянда, видеоигра, светильникпроектор, пирожное, сервер, bluetoothгарнитура, игрушкаантистресс, головоломка, блейзер, тесьма, мокасины, готовальня, клатч, ночник, фальшпогоны, пассатижиплоскогубцы, книгапазл, электрокуртка, гейнер, аудиокабель, парковка, топсайдеры, бронзер, пинетки, электроодеяло, погремушка, игрушкастучалка, лакомство, корни, фоамиран, эхолот, бронежилет, эспадрильи, вертушка, неглиже, подъюбник, ботфорты, берет, террариум, автомагнитола, спот, видеокабель, прорезыватель, клубни, дрельшуруповерт, дивандек, печенье,

In [19]:
# Номер кластера, для которого нужно найти главные слова
cluster_number = 61

# Проверяем, существует ли кластер с данным номером
if cluster_number in clustered_words:
    words_in_cluster = clustered_words[cluster_number]
    
    # Получаем векторы соответствующих слов
    vectors_in_cluster = [word_vectors[vocabulary.index(word)] for word in words_in_cluster]
    
    # Вычисляем центр кластера
    cluster_center = np.mean(vectors_in_cluster, axis=0)
    
    # Вычисляем расстояние от каждого слова до центра кластера
    distances = {word: np.linalg.norm(vector - cluster_center) for word, vector in zip(words_in_cluster, vectors_in_cluster)}
    
    # Находим три слова с наименьшим расстоянием до центра
    top_words = sorted(distances, key=distances.get)[:10]
    
    print(f"Самых главные слова в кластере {cluster_number}: {', '.join(top_words)}")
else:
    print(f"Кластер {cluster_number} не найден.")

Самых главные слова в кластере 61: литература, издание, художественная, книга, печатная


In [20]:
# Номер кластера, для которого нужно найти главные слова
cluster_number = 2

# Проверяем, существует ли кластер с данным номером
if cluster_number in clustered_words:
    words_in_cluster = clustered_words[cluster_number]
    
    # Получаем векторы соответствующих слов
    vectors_in_cluster = [word_vectors[vocabulary.index(word)] for word in words_in_cluster]
    
    # Вычисляем центр кластера
    cluster_center = np.mean(vectors_in_cluster, axis=0)
    
    # Вычисляем расстояние от каждого слова до центра кластера
    distances = {word: np.linalg.norm(vector - cluster_center) for word, vector in zip(words_in_cluster, vectors_in_cluster)}
    
    # Находим три слова с наименьшим расстоянием до центра
    top_words = sorted(distances, key=distances.get)[:10]
    
    print(f"Самые главные слова в кластере {cluster_number}: {', '.join(top_words)}")
else:
    print(f"Кластер {cluster_number} не найден.")

Самые главные слова в кластере 2: готовые, фрукты, десерты, рубаха, сушеная, водоросли, морские, копченая, шуба, охлажденная


In [21]:
# Номер кластера, для которого нужно найти главные слова
cluster_number = 4

# Проверяем, существует ли кластер с данным номером
if cluster_number in clustered_words:
    words_in_cluster = clustered_words[cluster_number]
    
    # Получаем векторы соответствующих слов
    vectors_in_cluster = [word_vectors[vocabulary.index(word)] for word in words_in_cluster]
    
    # Вычисляем центр кластера
    cluster_center = np.mean(vectors_in_cluster, axis=0)
    
    # Вычисляем расстояние от каждого слова до центра кластера
    distances = {word: np.linalg.norm(vector - cluster_center) for word, vector in zip(words_in_cluster, vectors_in_cluster)}
    
    # Находим три слова с наименьшим расстоянием до центра
    top_words = sorted(distances, key=distances.get)[:10]
    
    print(f"Самые главные слова в кластере {cluster_number}: {', '.join(top_words)}")
else:
    print(f"Кластер {cluster_number} не найден.")

Самые главные слова в кластере 4: картина


In [22]:
# Номер кластера, для которого нужно найти главные слова
cluster_number = 44

# Проверяем, существует ли кластер с данным номером
if cluster_number in clustered_words:
    words_in_cluster = clustered_words[cluster_number]
    
    # Получаем векторы соответствующих слов
    vectors_in_cluster = [word_vectors[vocabulary.index(word)] for word in words_in_cluster]
    
    # Вычисляем центр кластера
    cluster_center = np.mean(vectors_in_cluster, axis=0)
    
    # Вычисляем расстояние от каждого слова до центра кластера
    distances = {word: np.linalg.norm(vector - cluster_center) for word, vector in zip(words_in_cluster, vectors_in_cluster)}
    
    # Находим три слова с наименьшим расстоянием до центра
    top_words = sorted(distances, key=distances.get)[:10]
    
    print(f"Самые главные слова в кластере {cluster_number}: {', '.join(top_words)}")
else:
    print(f"Кластер {cluster_number} не найден.")

Самые главные слова в кластере 44: платье, пальто


In [23]:
# Номер кластера, для которого нужно найти главные слова
cluster_number = 58

# Проверяем, существует ли кластер с данным номером
if cluster_number in clustered_words:
    words_in_cluster = clustered_words[cluster_number]
    
    # Получаем векторы соответствующих слов
    vectors_in_cluster = [word_vectors[vocabulary.index(word)] for word in words_in_cluster]
    
    # Вычисляем центр кластера
    cluster_center = np.mean(vectors_in_cluster, axis=0)
    
    # Вычисляем расстояние от каждого слова до центра кластера
    distances = {word: np.linalg.norm(vector - cluster_center) for word, vector in zip(words_in_cluster, vectors_in_cluster)}
    
    # Находим три слова с наименьшим расстоянием до центра
    top_words = sorted(distances, key=distances.get)[:10]
    
    print(f"Самые главные слова в кластере {cluster_number}: {', '.join(top_words)}")
else:
    print(f"Кластер {cluster_number} не найден.")

Самые главные слова в кластере 58: шорты, носки, брюки


In [24]:
# Номер кластера, для которого нужно найти главные слова
cluster_number = 6

# Проверяем, существует ли кластер с данным номером
if cluster_number in clustered_words:
    words_in_cluster = clustered_words[cluster_number]
    
    # Получаем векторы соответствующих слов
    vectors_in_cluster = [word_vectors[vocabulary.index(word)] for word in words_in_cluster]
    
    # Вычисляем центр кластера
    cluster_center = np.mean(vectors_in_cluster, axis=0)
    
    # Вычисляем расстояние от каждого слова до центра кластера
    distances = {word: np.linalg.norm(vector - cluster_center) for word, vector in zip(words_in_cluster, vectors_in_cluster)}
    
    # Находим три слова с наименьшим расстоянием до центра
    top_words = sorted(distances, key=distances.get)[:10]
    
    print(f"Самые главные слова в кластере {cluster_number}: {', '.join(top_words)}")
else:
    print(f"Кластер {cluster_number} не найден.")

Самые главные слова в кластере 6: мусс, хна, ботокс, уксус, тинт, выпрямитель, мочесборник, гнездо, блеск, трубочки
